# Лабораторная работа №1

Задание: Используя Python и модули requests и re написать скрипт, извлекающий из веб-страницы(например mosigra.ru) все адреса электронной почты


In [1]:
# Импортируем модули

import requests
import re

### "Нарезка" на слова

In [2]:
# Функция, которая принимает на вход строку и нарезает ее на слова, состоящие из букв латинского алфавита 
# (кириллица нас не интересут, так как кириллические домены для почты все еще в тесте), цифр 0-9,  
# знаков '.', '-', '_' и '@', которые могут использоваться для имен почтовых ящиков.

# Функция возвращает list, элементами которого являются условно подходящие слова
# и убирает пробелы между словами

# Для удобства весь текст приводится к нижнему регистру

def cut_to_words(a): # передаем строку 
    a = a.lower()
    b = re.split('[^a-z^0-9@_.-]', a)
    
    #убираем из списка "пустые" слова
    x = list()
    for i in b:
        if i != '':
            x.append(i)        
        
    return x

### Анализ "ящик" или нет

In [3]:
# Функция получает на вход список слов, в которых можно "заподозрить" ящик
# (по сути, список всех выдернутых из текста страницы сайта слов)
# и возвращает список из слов, которые определила как "почтовый ящик"

def mailbox_or_not(x):
    
    # Согласно требованиям при создании ящиков от компании mail.ru (https://help.mail.ru/enmail-help/registration/name),
    # к имени постового ящика могут применятся следующие правила (!Внимание! правило о длине имени здесь игнорируется,
    # так как на других доменах такого ограничения часто нет): 
    # --- любые латинские символы (a-z, A-Z);
    # --- любые цифры (0-9);
    # --- специальные символы: ("-"), ("_") и (".") — данные символы не могут идти друг за другом, 
    #     а также быть первыми и последними символами в имени почтового ящика.

    
    x_new = list()
    j = 0

    for i in x:
        last = len(i) - 1    # номер последней буквы в слове, которое проверяем
        at = False           # наличие в слове ровно 1 буквы @
        #dot = False          # наличие знака . как минимум через один символ после @
    
        # если первый и последние символы не _./@
        if i[0] != '.' and i[0] != '_' and i[0] != '-' and i[last] != '.' and i[last] != '_' and i[last] != '-' and i[0] != '@' and i[last] != '@': 
            for k in range(last):
                if i[k] == '@':
                    at = True
                
                    # продолжаем искать @ после первой найденной в слове
                    t = k + 1
                    while t <= last:    
                        if i[t] == '@':
                            at = False    # если в слове нашлась хотя бы еще одна @, то отбраковываем
                            break
                        t += 1    
                
                    if at == False:
                        break
                    else:
                        # Адрес может выгядеть как xxxxx@xxxx.xx или xxxxx@xxxx.xxx, xxxx@xxxx.xxxx (.it, .ru, .org, .info)
                        # Поэтому отбираем слова, где полсле знака @ сначала идет ХОТЯ БЫ один разрешенный символ, а не точка,
                        # но есть точка, ХОТЯ бы за ДВА символа до конца слова
                                   
                        if i[k + 1] != '.' and i[k + 1] != '_' and i[k + 1] != '-':
                            t = k + 2
                            while t <= last:    
                                if i[t] == '.' and t != (last -1):
                                    #dot = True
                                    x_new.append(i)
                                #    print(x_new[j])
                                    j += 1
                                    break
                                t += 1    
                    
    return x_new


### Убирает дублирующиеся ящики

In [4]:
def delete_copy(d):
    d_new = list()
    d_new.append(d[0])

    for k in d:  # для каждого элемента списка    
            # Кажется, в Python 3 нет этой функции:
            # if dict_of_mailadresses.has_key(k) == False:
            # А жаль...
        
            marker = 0        
            for l in d_new:
                if l == k: 
                    marker += 1  # слова нет в новом списке один раз, если marker == 0
                                 # слово есть в новом списке, если marker > 0
            if marker == 0:
                d_new.append(k)

    return d_new

In [6]:
# Получить имя сайта, по которому нужна информация
print('Enter sitename, please')

# Введите имя сайта в формате http://... или www...
# Примеры, чтобы не искать: https://geektimes.ru/post/74326/  
# - http://www.mosigra.ru/  
# - http://mlbootcamp.ru/article/tutorial/ 
# - https://pypi.python.org/pypi/requests/2.11.1

sitename = input()
#sitename = 'http://www.mosigra.ru/'  #для тестов, чтобы постоянно не вводить имя сайта

# Сделаем минимальную проверку, что введено имя сайта
check_site_name = True

# В этом месте можно добавить более сложне проверки
if sitename[:8] != 'https://' and sitename[:7] != 'http://':
    check_site_name = False
    print('The site name is wrong')
else:
    print('Please, wait a minute...')
    r = requests.get(sitename)
         
    sitecontent = r.text    # в эту СТРОКУ запишем содержимое, взятое с исследуемого сайта
    
    # Уберем все лишнее, создав список только из "подозрительных на почту" слов
    sitecontent_by_words = cut_to_words(sitecontent)
    
    # Проанализируем слова из списка на предмет, являются ли они похожими на почтовый ящик. 
    # Получившийся на этом шаге список содержит только имена почтовых ящиков.
    mailboxes = mailbox_or_not(sitecontent_by_words)      
    
    if mailboxes != []:
       
        # Уберем из списка почтовых ящиков все дубликаты
        mailboxes = delete_copy(mailboxes)
    
        print('On the page of site ', sitename, ' there are e-mailaddresse:\n')
        print('\n'.join(mailboxes))
    else:
        print('No one e-mail address on this page. Sorry')

Enter sitename, please
htp://www.mosigra.ru/
The site name is wrong
